In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Import required libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
import torchvision
import torchvision.models as models
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from scipy.io import loadmat
import matplotlib.pyplot as plt
import random
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import datetime
import time
import numpy as np 
import pandas as pd 

In [2]:
class CarsDataset(Dataset):
    def __init__(self, annotations, img_dir, transform=None):
        self.img_paths = [os.path.join(img_dir, sample[-1][0]) for sample in annotations[0]]
        self.labels = [sample[-2][0][0] - 1 for sample in annotations[0]]
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        image = Image.open(self.img_paths[idx]).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
        
        return image, label

cars_dataset = torch.load("cars196_dataset.pt", weights_only=False)

In [3]:
# הצגת גודל הדאטהסט
print(f"מספר דוגמאות בדאטהסט: {len(cars_dataset)}")


מספר דוגמאות בדאטהסט: 16185


In [4]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),  # הופכים תמונה אופקית באופן אקראי
    transforms.RandomRotation(15),  # מסובבים מעט את התמונה כדי להוסיף גיוון
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # התאמות צבע
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# עבור בדיקות - לא צריך אוגמנטציה
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
# חלוקת הדאטה לאימון ולמבחן
dataset_size = len(cars_dataset)
train_size = int(0.7 * dataset_size)  # 70% מהדאטה
test_size = dataset_size - train_size  # 30% הנותרים

train_indices, test_indices = train_test_split(range(dataset_size), train_size=train_size, test_size=test_size, random_state=42)

train_dataset = Subset(cars_dataset, train_indices)
test_dataset = Subset(cars_dataset, test_indices)

In [6]:
# יצירת DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [7]:
from torchvision.models import MobileNet_V2_Weights

base_model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)


In [8]:
class CustomMobileNet(nn.Module):
    def __init__(self, base_model, num_classes=196):
        super(CustomMobileNet, self).__init__()
        self.base = base_model.features  # שמירה על כל השכבות של MobileNetV2
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)  # pooling
        self.dropout = nn.Dropout(0.5)  # הוסף שכבת Dropout
        self.fc = nn.Linear(1280, num_classes)

    def forward(self, x):
        x = self.base(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)  # Flatten לפני fully connected layers
        x = self.dropout(x)  # Dropout לפני השכבה הסופית
        x = self.fc(x)
        return x

# יצירת המודל עם 196 מחלקות
model = CustomMobileNet(base_model, num_classes=196)

In [9]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model.to(device)

CustomMobileNet(
  (base): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [10]:
dummy_input = torch.randn(1, 3, 224, 224)  # תמונה מדומה
output = model(dummy_input)
print("Output shape:", output.shape)  # צריך להיות torch.Size([1, 196])


Output shape: torch.Size([1, 196])


In [11]:
from torchinfo import summary

summary(model, input_size=(1, 3, 224, 224), col_names=["input_size", "output_size", "num_params", "trainable"])


Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Trainable
CustomMobileNet                                    [1, 3, 224, 224]          [1, 196]                  --                        True
├─Sequential: 1-1                                  [1, 3, 224, 224]          [1, 1280, 7, 7]           --                        True
│    └─Conv2dNormActivation: 2-1                   [1, 3, 224, 224]          [1, 32, 112, 112]         --                        True
│    │    └─Conv2d: 3-1                            [1, 3, 224, 224]          [1, 32, 112, 112]         864                       True
│    │    └─BatchNorm2d: 3-2                       [1, 32, 112, 112]         [1, 32, 112, 112]         64                        True
│    │    └─ReLU6: 3-3                             [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
│    └─InvertedResidual: 2-2                       [1, 32, 

In [12]:
for param in model.parameters():
    param.requires_grad = False  # מקפיא את כל השכבות

for param in list(model.base.parameters())[-4:]:  
    param.requires_grad = True  # פותח 4 השכבות האחרונות לאימון

In [13]:
print(f"Train Loader Size: {len(train_loader.dataset)}")
print(f"Test Loader Size: {len(test_loader.dataset)}")


Train Loader Size: 11329
Test Loader Size: 4856


In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [15]:
def train_epoch(model, train_loader, criterion, optimizer, scheduler):
    model.train()
    model.to(device)
    running_loss = 0
    
    print("Starting Training Epoch...")  # בדיקה

    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data = data.to("cpu")
        target = target.to("cpu").long()
        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()

    running_loss /= len(train_loader)
    scheduler.step()
    
    print('Train Loss:', running_loss)
    return running_loss


In [16]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        model.to(device)
        running_loss = 0
        total_predictions = 0
        correct_predictions = 0

        # CODE HERE
        for batch_idx,(data,target) in enumerate(test_loader):
            data = data.to("cpu")
            target = target.to("cpu").long()
            outputs = model(data) #forward step
            _, predicted = torch.max(outputs.data,1)
            total_predictions += target.size(0)
            correct_predictions += (predicted==target).sum().item()
            
            loss = criterion(outputs,target)#loss
            running_loss += loss.item()
            
        running_loss /= len(train_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Test Loss: ',running_loss, ' Accuracy: ', acc,'%')
        return running_loss,acc

In [17]:
n_epochs = 20
Train_loss = []
Test_loss = []
Test_acc = []

try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))


Starting Training Epoch...
Train Loss: 4.98505788326936
Test Loss:  1.9387338632252724  Accuracy:  16.186161449752884 %
************************************************************
Starting Training Epoch...
Train Loss: 4.3723437019396565
Test Loss:  1.7682771887866666  Accuracy:  24.794069192751238 %
************************************************************
Starting Training Epoch...
Train Loss: 4.052766601524837
Test Loss:  1.6848998880184587  Accuracy:  29.2833607907743 %
************************************************************
Starting Training Epoch...
Train Loss: 3.838150596753162
Test Loss:  1.6229099474101207  Accuracy:  32.96952224052718 %
************************************************************
Starting Training Epoch...
Train Loss: 3.676766118814982
Test Loss:  1.5718680951423807  Accuracy:  35.09060955518946 %
************************************************************
Starting Training Epoch...
Train Loss: 3.5241305764202338
Test Loss:  1.5600598231693585  Accu

In [18]:
## more layers

In [19]:
for param in model.parameters():
    param.requires_grad = False  # מקפיא את כל השכבות

for param in list(model.base.parameters())[-8:]:  
    param.requires_grad = True  # פותח 8 השכבות האחרונות לאימון

In [20]:
n_epochs = 10
Train_loss = []
Test_loss = []
Test_acc = []

try:
    for i in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)
        
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))

Starting Training Epoch...
Train Loss: 3.0345157080544065
Test Loss:  1.4143547608250455  Accuracy:  42.40115321252059 %
************************************************************
Starting Training Epoch...
Train Loss: 3.014421894284667
Test Loss:  1.4113532427503965  Accuracy:  41.989291598023065 %
************************************************************
Starting Training Epoch...
Train Loss: 2.9942449185004185
Test Loss:  1.4017895075094549  Accuracy:  42.27759472817134 %
************************************************************
Starting Training Epoch...
Train Loss: 2.9729334443849638
Test Loss:  1.3931486952960574  Accuracy:  42.46293245469522 %
************************************************************
Starting Training Epoch...
Train Loss: 2.9481275182516855
Test Loss:  1.3774592519983417  Accuracy:  43.266062602965405 %
************************************************************
Starting Training Epoch...
Train Loss: 2.9227969088238286
Test Loss:  1.3842583200322893 

In [21]:
torch.save(model.state_dict(), "test_3.pth")
print("Model saved successfully.")

Model saved successfully.


In [22]:
# עדכון קצב הלמידה - נקטין ב-50%
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.5  # מקטין פי 2

print(f" קצב הלמידה החדש: {optimizer.param_groups[0]['lr']:.6f}")

# אימון נוסף עם 10 אפוקים
n_epochs_continue = 10
try:
    for i in range(n_epochs_continue):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)

        print(f"Epoch {i+1}/{n_epochs_continue} - LR: {optimizer.param_groups[0]['lr']:.6f}")
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))


 קצב הלמידה החדש: 0.000002
Starting Training Epoch...
Train Loss: 2.881522145695007
Test Loss:  1.3700622755650573  Accuracy:  44.501647446457994 %
Epoch 1/10 - LR: 0.000002
************************************************************
Starting Training Epoch...
Train Loss: 2.881600156321344
Test Loss:  1.373570790916303  Accuracy:  44.02800658978583 %
Epoch 2/10 - LR: 0.000002
************************************************************
Starting Training Epoch...
Train Loss: 2.8763106306449986
Test Loss:  1.3696628178460641  Accuracy:  43.5337726523888 %
Epoch 3/10 - LR: 0.000002
************************************************************
Starting Training Epoch...
Train Loss: 2.8775617847657844
Test Loss:  1.3702311986592324  Accuracy:  43.121911037891266 %
Epoch 4/10 - LR: 0.000002
************************************************************
Starting Training Epoch...
Train Loss: 2.871445537117876
Test Loss:  1.3695443695455747  Accuracy:  43.76029654036244 %
Epoch 5/10 - LR: 0.0000

In [23]:
torch.save(model.state_dict(), "test_4.pth")
print("Model saved successfully.")

Model saved successfully.


✔ אם ה-Test Loss לא יורד משמעותית והדיוק לא משתפר יותר.
✔ אם ה-LR הנוכחי מאוד נמוך (למשל 0.000009 כמו שהיה לך).
✔ אם המודל עוד לא הגיע ל-Overfitting (ה-Train Loss ו-Test Loss דומים).

In [25]:
# העלאת קצב הלמידה כדי לאפשר למודל ללמוד מהר יותר
for param_group in optimizer.param_groups:
    param_group['lr'] = 0.00005  # הגדלת קצב הלמידה
print(f" קצב הלמידה החדש: {optimizer.param_groups[0]['lr']:.6f}")


 קצב הלמידה החדש: 0.000050


In [26]:
# אימון נוסף עם 10 אפוקים
n_epochs_continue = 10
try:
    for i in range(n_epochs_continue):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, scheduler)  
        test_loss, test_acc = test_model(model, test_loader, criterion)
        
        Train_loss.append(train_loss)
        Test_loss.append(test_loss)
        Test_acc.append(test_acc)

        print(f"Epoch {i+1}/{n_epochs_continue} - LR: {optimizer.param_groups[0]['lr']:.6f}")
        print('*' * 60)

except Exception as e:
    print("Error during training:", str(e))


Starting Training Epoch...
Train Loss: 2.897140506964981
Test Loss:  1.3510410382145719  Accuracy:  44.64579901153213 %
Epoch 1/10 - LR: 0.000050
************************************************************
Starting Training Epoch...
Train Loss: 2.860440217221237
Test Loss:  1.3494486018531917  Accuracy:  45.716639209225704 %
Epoch 2/10 - LR: 0.000050
************************************************************
Starting Training Epoch...
Train Loss: 2.8197179889813464
Test Loss:  1.3443392409256045  Accuracy:  45.11943986820428 %
Epoch 3/10 - LR: 0.000050
************************************************************
Starting Training Epoch...
Train Loss: 2.790063894013592
Test Loss:  1.3342764895456634  Accuracy:  45.42833607907743 %
Epoch 4/10 - LR: 0.000050
************************************************************
Starting Training Epoch...
Train Loss: 2.7610946802897915
Test Loss:  1.3261105812487044  Accuracy:  45.94316309719934 %
Epoch 5/10 - LR: 0.000025
***********************

In [27]:
#####

In [28]:
## now with more Augmentation 

In [29]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.75, 1.0)),  # זום אקראי
    transforms.RandomHorizontalFlip(p=0.5),  # הפיכה אופקית
    transforms.RandomRotation(degrees=30),  # סיבוב חזק יותר
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # התאמות צבע
    transforms.RandomAffine(degrees=0, shear=10, scale=(0.9, 1.1)),  # עיוות גזירה (Shear)
    transforms.RandomErasing(p=0.3),  # מחיקה רנדומלית להוספת רעש
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [30]:

# חלוקת הדאטה לאימון ולמבחן (כמו קודם)
dataset_size = len(cars_dataset)
train_size = int(0.7 * dataset_size)  # 70% לאימון
test_size = dataset_size - train_size  # 30% למבחן

train_indices, test_indices = train_test_split(range(dataset_size), train_size=train_size, test_size=test_size, random_state=42)

# יצירת תת-סטים עם Augmentation מתאים
train_dataset = Subset(cars_dataset, train_indices)
test_dataset = Subset(cars_dataset, test_indices)

# עדכון הטרנספורמציות
train_dataset.dataset.transform = train_transform
test_dataset.dataset.transform = test_transform

# יצירת DataLoader
batch_size = 32  # if it work
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)


